In [3]:
import sys
sys.path.append("..")
import os

from src.image_processing.image_utils import convert_pdf_to_png
from src.image_processing.ocr import process_png_images
from src.text_processing.text_utils import concatenate_text_files
from src.text_processing.reasoning import create_tex
from src.pdf_generation.pdf_creator import output_pdf

api_key = os.getenv('OPENAI_API_KEY')

In [4]:
# SET THE NAME OF THE FILE
file_name = "aula13"

In [5]:
# Turn PDF into separate PNGs
convert_pdf_to_png(file_name)

# Extract text from image
process_png_images(file_name)

# Concatenate descriptions
concatenate_text_files(file_name)

# Create tex file
create_tex(file_name)

# Output PDF
output_pdf(file_name)

../data/processed\aula13\pngs\1.png
Processed and saved OCR text for: 1.png
../data/processed\aula13\pngs\2.png
Processed and saved OCR text for: 2.png
All text files have been concatenated into '../data/processed\aula13\combined_text.txt'.
TOTAL TOKENS: 623
Formatted notes saved to ../data/processed\aula13\formatted_notes.tex


[WARNING] Unusual conversion: to convert a .tex file to PDF, you get better results by using pdflatex (or lualatex or xelatex) directly, try `pdflatex ../data/processed\aula13\formatted_notes.tex` instead of `pandoc ../data/processed\aula13\formatted_notes.tex -o ../data/processed\aula13\formatted_notes.pdf`.
pdflatex: major issue: So far, you have not checked for MiKTeX updates.
pdflatex: major issue: So far, you have not checked for MiKTeX updates.



PDF successfully created at ../data/processed\aula13\formatted_notes.pdf
PDF generated successfully.
